# **_PRÁCTICA 3 - TAA_**

_Pablo Martín de Benito_

***

## **_Parte 1_**

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
import six
import sys
sys.modules['sklearn.externals.six'] = six
# Estos tres modulos son imprescindibles para el ID3
from id3 import Id3Estimator, export_graphviz

In [4]:
tabla = pd.DataFrame(columns=['Presion','Clase'],index=np.arange(6))
tabla.Presion = [40,48,60,72,80,90]
tabla.Clase = ['+','+','-','-','-','+']
tabla

,Presion,Clase
0,40,+
1,48,+
2,60,-
3,72,-
4,80,-
5,90,+


In [5]:
tabla.Clase.value_counts(normalize=True)

Clase
+    0.5
-    0.5
Name: proportion, dtype: float64

In [6]:
# No se puede copiar como si fueran variables tabla_nueva = tabla_antigua
tabla1 = tabla.copy()
tabla1.Presion = np.digitize(tabla.Presion, bins=[(48+60)/2])   # bins  ->  Conjunto de cotas a partir del cual se establecen los primeros tramos
tabla1

,Presion,Clase
0,0,+
1,0,+
2,1,-
3,1,-
4,1,-
5,1,+


In [7]:
H = entropy(tabla1.iloc[:,-1].value_counts(normalize=True), base=2)

l1 = len(tabla1)
G1 = H
for v in tabla1.Presion.unique():
    Class1 = tabla1.loc[tabla1.Presion == v]
    # print(table)
    G1 -= (len(Class1)/l1)*entropy(Class1.iloc[:,-1].value_counts(normalize=True), base =2)
G1

0.4591479170272448

In [8]:
tabla2 = tabla.copy()
tabla.Presion = np.digitize(tabla.Presion, bins=[(80+90)/2])
tabla2


,Presion,Clase
0,40,+
1,48,+
2,60,-
3,72,-
4,80,-
5,90,+


In [9]:
l2 = len(tabla2)
G2 = H
for v in tabla2.Presion.unique():
    Class2 = tabla2.loc[tabla2.Presion == v]
    # print(table)
    G2 -= (len(Class2)/l2)*entropy(Class2.iloc[:,-1].value_counts(normalize=True), base =2)
G2

1.0

In [10]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree, export_text

In [11]:
clf = DecisionTreeClassifier(criterion='entropy', max_leaf_nodes = 2)

In [12]:
DecisionTreeClassifier?

Init signature:
DecisionTreeClassifier(
    *,
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0,
)
Docstring:     
A decision tree classifier.

Read more in the :ref:`User Guide <tree>`.

Parameters
----------
criterion : {"gini", "entropy", "log_loss"}, default="gini"
    The function to measure the quality of a split. Supported criteria are
    "gini" for the Gini impurity and "log_loss" and "entropy" both for the
    Shannon information gain, see :ref:`tree_mathematical_formulation`.

splitter : {"best", "random"}, default="best"
    The strategy used to choose the split at each node. Supported
    strategies are "best" to choose the best split and "random" to choose
    the best random split.

max_depth : int, default=None
    The maximum depth o

In [13]:
clf.fit(np.array(tabla.Presion).reshape(-1,1), tabla.Clase)

DecisionTreeClassifier(criterion='entropy', max_leaf_nodes=2)

In [14]:
txt = export_text(clf, feature_names=['Presion'])
print(txt)

|--- Presion <= 0.50
|   |--- class: -
|--- Presion >  0.50
|   |--- class: +



***

## **_Parte 2_**


In [15]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
thoracic_surgery_data = fetch_ucirepo(id=277) 
  
# data (as pandas dataframes) 
X = thoracic_surgery_data.data.features 
y = thoracic_surgery_data.data.targets 
  
# metadata 
#print(thoracic_surgery_data.metadata) 
  
# variable information 
#print(thoracic_surgery_data.variables) 

In [16]:
print(X)

      DGN  PRE4  PRE5  PRE6 PRE7 PRE8 PRE9 PRE10 PRE11 PRE14 PRE17 PRE19  \
0    DGN2  2.88  2.16  PRZ1    F    F    F     T     T  OC14     F     F   
1    DGN3  3.40  1.88  PRZ0    F    F    F     F     F  OC12     F     F   
2    DGN3  2.76  2.08  PRZ1    F    F    F     T     F  OC11     F     F   
3    DGN3  3.68  3.04  PRZ0    F    F    F     F     F  OC11     F     F   
4    DGN3  2.44  0.96  PRZ2    F    T    F     T     T  OC11     F     F   
..    ...   ...   ...   ...  ...  ...  ...   ...   ...   ...   ...   ...   
465  DGN2  3.88  2.12  PRZ1    F    F    F     T     F  OC13     F     F   
466  DGN3  3.76  3.12  PRZ0    F    F    F     F     F  OC11     F     F   
467  DGN3  3.04  2.08  PRZ1    F    F    F     T     F  OC13     F     F   
468  DGN3  1.96  1.68  PRZ1    F    F    F     T     T  OC12     F     F   
469  DGN3  4.72  3.56  PRZ0    F    F    F     F     F  OC12     F     F   

    PRE25 PRE30 PRE32  AGE  
0       F     T     F   60  
1       F     T     F   51  


In [17]:
print(y)

    Risk1Yr
0         F
1         F
2         F
3         F
4         T
..      ...
465       F
466       F
467       F
468       F
469       F

[470 rows x 1 columns]


### **_Pregunta 1 - ¿Por qué no podemos aplicar el algoritmo ID3 directamente?_**
Porque en el conjunto de datos contamos con atributos que tienen valores continuos mientras que el algoritmo ID3 trabaja con valores discretos, para poder aplicarlo habría que discretizar dichos atributos.

### **Análisis con todos los atributos**

In [18]:
y.iloc[:,0].values.shape

(470,)

In [19]:
y.columns[0]

'Risk1Yr'

In [20]:
tabla_csv = X.copy()
tabla_csv.insert(loc=len(X.columns), column=y.columns[0], value=y)
tabla_csv.to_csv("thoracic.csv", index=False)
tabla_csv

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,DGN2,2.88,2.16,PRZ1,F,F,F,T,T,OC14,F,F,F,T,F,60,F
1,DGN3,3.40,1.88,PRZ0,F,F,F,F,F,OC12,F,F,F,T,F,51,F
2,DGN3,2.76,2.08,PRZ1,F,F,F,T,F,OC11,F,F,F,T,F,59,F
3,DGN3,3.68,3.04,PRZ0,F,F,F,F,F,OC11,F,F,F,F,F,54,F
4,DGN3,2.44,0.96,PRZ2,F,T,F,T,T,OC11,F,F,F,T,F,73,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,DGN2,3.88,2.12,PRZ1,F,F,F,T,F,OC13,F,F,F,T,F,63,F
466,DGN3,3.76,3.12,PRZ0,F,F,F,F,F,OC11,F,F,F,T,F,61,F
467,DGN3,3.04,2.08,PRZ1,F,F,F,T,F,OC13,F,F,F,F,F,52,F
468,DGN3,1.96,1.68,PRZ1,F,F,F,T,T,OC12,F,F,F,T,F,79,F


In [21]:
f = open('thoracic_tree.txt')
contenido = f.read()
f.close()
# Para poder ver bien el contenido tenemos que ponerle print()

In [22]:
print(contenido)

PRE9 = F
|   PRE17 = F
|   |   DGN = DGN2
|   |   |   PRE30 = T
|   |   |   |   PRE10 = T
|   |   |   |   |   PRE6 = PRZ1
|   |   |   |   |   |   PRE14 = OC14: F (3.0)
|   |   |   |   |   |   PRE14 = OC12
|   |   |   |   |   |   |   PRE8 = F: T (6.0/2.0)
|   |   |   |   |   |   |   PRE8 = T: F (6.0/1.0)
|   |   |   |   |   |   PRE14 = OC11: F (5.0/1.0)
|   |   |   |   |   |   PRE14 = OC13: F (2.0)
|   |   |   |   |   PRE6 = PRZ0: T (2.0)
|   |   |   |   |   PRE6 = PRZ2: F (4.0/1.0)
|   |   |   |   PRE10 = F: F (10.0/1.0)
|   |   |   PRE30 = F: F (6.0)
|   |   DGN = DGN3
|   |   |   PRE7 = F
|   |   |   |   PRE14 = OC14: F (7.0/3.0)
|   |   |   |   PRE14 = OC12
|   |   |   |   |   PRE30 = T
|   |   |   |   |   |   PRE10 = T
|   |   |   |   |   |   |   PRE6 = PRZ1
|   |   |   |   |   |   |   |   PRE25 = F
|   |   |   |   |   |   |   |   |   PRE11 = T: F (16.0/1.0)
|   |   |   |   |   |   |   |   |   PRE11 = F
|   |   |   |   |   |   |   |   |   |   PRE8 = F: F (67.0/10.0)
|   |   |   |  

Correctly Classified Instances         123               76.875  %

Incorrectly Classified Instances        37               23.125  %

### **Ver si alguno de los atributos no se usa**


In [23]:
no_usado = []
for atr in tabla_csv.columns[:-1]:
    if contenido.find(atr) == -1:
        no_usado.append(atr)
no_usado

['PRE19', 'PRE32']

#### **Eliminamos dichos atributos y volvemos a realizar el analisis**

In [24]:
tabla_csv.drop(labels=no_usado, axis=1, inplace=True) # axis=1 por columnas
# inplace = True  ->  Si no ponemos el inplace, no hace el borrado, lo hace en la primera impresion pero no en realidad, si lo vuelves a imprimir no lo ha borrado.


In [25]:
tabla_csv.to_csv('ThoracicSurgery_filtrado_atributos.csv', index = False)

Correctly Classified Instances         123               76.875  %

Incorrectly Classified Instances        37               23.125  %

### **Discretización de los atributos numéricos: ['PRE4','PRE5',''AGE]**

In [26]:
Atrib_Num = list(tabla_csv._get_numeric_data().columns)


In [27]:
import re
for atributo in Atrib_Num:
    b = []
    for pos in re.finditer(atributo+' <= ', contenido):
        if contenido[pos.end():].find(":") < contenido[pos.end():].find("\n"):
            segmento = contenido[pos.end():].find(":")
        else:
            segmento = contenido[pos.end():].find("\n")
        b.append(float(contenido[pos.end():pos.end()+segmento]))
        print(b[-1])
    c = list(set(b))
    c.sort(key=float)
    print(atributo,c)
    tabla_csv[atributo] = [chr(65+i) for i in np.digitize(tabla_csv[atributo], bins=c)]
    tabla_csv

2.88
2.66
PRE4 [2.66, 2.88]
2.04
PRE5 [2.04]
70.0
65.0
62.0
AGE [62.0, 65.0, 70.0]


In [28]:
tabla_csv

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE25,PRE30,AGE,Risk1Yr
0,DGN2,C,B,PRZ1,F,F,F,T,T,OC14,F,F,T,A,F
1,DGN3,C,A,PRZ0,F,F,F,F,F,OC12,F,F,T,A,F
2,DGN3,B,B,PRZ1,F,F,F,T,F,OC11,F,F,T,A,F
3,DGN3,C,B,PRZ0,F,F,F,F,F,OC11,F,F,F,A,F
4,DGN3,A,A,PRZ2,F,T,F,T,T,OC11,F,F,T,D,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,DGN2,C,B,PRZ1,F,F,F,T,F,OC13,F,F,T,B,F
466,DGN3,C,B,PRZ0,F,F,F,F,F,OC11,F,F,T,A,F
467,DGN3,C,B,PRZ1,F,F,F,T,F,OC13,F,F,F,A,F
468,DGN3,A,A,PRZ1,F,F,F,T,T,OC12,F,F,T,D,F


In [29]:
tabla_csv.to_csv('Thoracic_Nominal.csv', index = False)

Correctly Classified Instances         130               81.25   %

Incorrectly Classified Instances        30               18.75   %